### Import modules

### 0. Helper Functions

In [11]:
import helperFunctions
import hyperparameterTuning
from importAndSplitting import prepare_datasets
import sanityCheck
import train
import transferLearning
from primaryModel import YogaClassfier

### 1. Import the data and Splitting

In [9]:
data_dir = '/Users/harrynguyen/Documents/GitHub/APS360_Yoga_Poses_Classification/Dataset/dataset'
train_dir = '/Users/harrynguyen/Documents/GitHub/APS360_Yoga_Poses_Classification/BaselineModel/yoga_train'
val_dir = '/Users/harrynguyen/Documents/GitHub/APS360_Yoga_Poses_Classification/BaselineModel/yoga_val'
test_dir = '/Users/harrynguyen/Documents/GitHub/APS360_Yoga_Poses_Classification/BaselineModel/yoga_test'

prepare_datasets(data_dir, train_dir, val_dir, test_dir)

Number of training images: 3838
Number of validation images: 872
Number of test images: 975


### 2. Build CNN

In [12]:
primary_model = YogaClassfier(num_classes=107)

### 3. Train CNN

### 4. Sanity check

In [16]:
source_dir = '/Users/harrynguyen/Documents/GitHub/APS360_Yoga_Poses_Classification/Dataset/dataset'
target_dir = '/Users/harrynguyen/Documents/GitHub/APS360_Yoga_Poses_Classification/BaselineModel/small_dataset'
sanityCheck.create_small_dataset(source_dir, target_dir)

AttributeError: module 'sanityCheck' has no attribute 'create_small_dataset'

In [ ]:
small_model = YogaClassfier()
small_model, train_err, train_loss = sanityCheck.train_on_small_dataset(small_model)

### 5. Hyperparameter Tuning

### 6. Tranfer Learning